In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from models import HuBERTFeatureFusion
import pandas as pd

import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

from torchview import draw_graph
import torchvision
# from torchviz import make_dot
from torchsummary import summary

# import torch
# torch.autograd.detect_anomaly(True)

/home/maximos/miniconda3/envs/torchaudio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_audio_folder = '/media/maindisk/maximos/data/melos_singers/Rebetika_vowels/train/'
csv_path = '/media/maindisk/maximos/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)
print(features_list)
del(features_list[:2])
print(features_list)

['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity']
['singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', '

In [4]:
task_labels_num_out = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_num_out[features_list[i]] = 1
# add singer identification
task_labels_num_out['singer_id'] = feats['singer_id'].max()+1 # accounting for zero
print(task_labels_num_out)

{'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'singer_id': 6}


In [5]:
model = HuBERTFeatureFusion(task_labels_num_out=task_labels_num_out)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [7]:
print(len(training_data))

336


In [8]:
print(training_data[235])

{'input_values': array([-7.37490505e-02, -1.50222912e-01, -1.27510890e-01, -1.06018946e-01,
       -6.70360550e-02, -4.32924852e-02, -1.57288127e-02,  9.61929560e-04,
        3.01078763e-02,  4.92679887e-02,  6.46991432e-02,  6.99052066e-02,
        6.00759089e-02,  4.65988480e-02,  3.56184877e-02,  3.16383652e-02,
        3.53685617e-02,  3.78059670e-02,  3.32244486e-02,  2.13740580e-02,
       -6.73217652e-03, -3.85746621e-02, -6.69399872e-02, -8.17108005e-02,
       -8.16208422e-02, -7.93872848e-02, -8.07959288e-02, -7.15225041e-02,
       -4.69668508e-02, -6.45332132e-03,  3.62963490e-02,  6.31492659e-02,
        6.80516362e-02,  6.10178113e-02,  5.03115058e-02,  4.44968790e-02,
        4.05212305e-02,  3.42018977e-02,  2.39775907e-02,  7.78215891e-03,
       -8.74485821e-03, -1.70489121e-02, -1.83677692e-02, -1.88833699e-02,
       -2.21180804e-02, -2.75980867e-02, -3.11374404e-02, -3.00933998e-02,
       -2.59017572e-02, -1.76573135e-02, -4.49645845e-03,  1.14213619e-02,
        

In [9]:
dataloader = DataLoader(training_data, batch_size=4, shuffle=True, collate_fn=model.collate_fn)

In [10]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x771d8474c050>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 4, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x771cbcec0290>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x771cbce003b0>, 'generator': None, 'collate_fn': <bound method HuBERTFeatureFusion.collate_fn of HuBERTFeatureFusion(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False

In [11]:
b = next(iter(dataloader))

In [12]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))
print(b[1])

4
4
33
{'Unnamed: 0': [221, 139, 198, 336], 'names': ['keti_grei_a_4', 'kazantzidis_old_e_4', 'kazantzidis_w_19', 'tsaousakis_i_10'], 'singer_id': [3, 2, 2, 5], 'Pitch': [0.293339640678077, 0.1864611559634963, 0.23071620513742888, 0.097297674946266], 'SpectralCentroid': [0.5673079703905136, 0.39263221346266125, 0.23657958573042165, 0.19106775408006565], 'SpectralSpread': [0.5932102147634521, 0.4824124261944179, 0.3241347217986273, 0.4225197145560895], 'SpectralSkewness': [0.28059685476284274, 0.30941813653296985, 0.6487231712083141, 0.44436298467581653], 'SpectralKurtosis': [0.029849123495943507, 0.0642965058457786, 0.3542313056605466, 0.1157268081710951], 'SpectralFlatness': [0.06352955020705191, 0.04139388668788378, 1.9822018235466833e-06, 2.6086288122556797e-06], 'SpectralCrest': [0.12311954433670846, 0.1278219782760998, 0.223313456925111, 0.5288145477594663], 'SpectralSlope': [0.4412898935203473, 0.4065613033400427, 0.5066140164888842, 0.472748157080816], 'SpectralDecrease': [0.120

In [13]:
# model.train()
# model.eval()
for p in model.hubert.parameters():
    p.requires_grad = False
train_params = []
for k in model.projectors.keys():
    train_params.extend( list(model.projectors[k].parameters()) )
    train_params.extend( list(model.classifiers[k].parameters()) )
    print(k)
    model.projectors[k].requires_grad = True
    model.classifiers[k].requires_grad = True
# # model.train(False)
# print(train_params)
# optimizer = Adam( model.classifiers['Pitch'].parameters() )
optimizer = Adam( model.parameters(), lr=0.001 )

Pitch
SpectralCentroid
SpectralSpread
SpectralSkewness
SpectralKurtosis
SpectralFlatness
SpectralCrest
SpectralSlope
SpectralDecrease
SpectralRollOff
SpectralVariation
SpectralFlux
HarmonicSpectralDeviation
Tristimulus_1
Tristimulus_2
Tristimulus_3
HarmonicOddToEvenRatio
Inharmonicity
HarmonicEnergy
NoiseEnergy
Noisiness
HarmonicToNoiseEnergy
PartialsToNoiseEnergy
F1_Hz
F2_Hz
F3_HZ
F4_Hz
singer_id


In [14]:
# for p in model.parameters():
#     print(p)
print(model.parameters())

<generator object Module.parameters at 0x771cbcf174c0>


In [15]:
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [16]:
print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

None


In [17]:
b[0]['attention_mask']

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [18]:
print(b[1]['singer_id'])

[3, 2, 2, 5]


In [19]:
y = model(
    audio_normalized=b[0]['input_values'],
    attention_mask=b[0]['attention_mask'],
    labels=b[1],
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True
)

In [20]:
print(y)

SequenceClassifierOutput(loss=tensor(3.9129, device='cuda:0', grad_fn=<AddBackward0>), logits={'Pitch': [tensor([[0.4855],
        [0.4850],
        [0.4815],
        [0.4826]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralCentroid': [tensor([[0.5032],
        [0.5026],
        [0.5056],
        [0.5036]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralSpread': [tensor([[0.5086],
        [0.5092],
        [0.5077],
        [0.5081]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralSkewness': [tensor([[0.5052],
        [0.5060],
        [0.5034],
        [0.5047]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralKurtosis': [tensor([[0.5009],
        [0.5101],
        [0.5028],
        [0.5023]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralFlatness': [tensor([[0.5136],
        [0.5137],
        [0.5105],
        [0.5140]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralCrest': [tensor([[0.4792],
        [0.4827],
        [0.4810],


In [21]:
print(y['logits'].keys())
for k in y['logits'].keys():
    print(y['logits'][k])

dict_keys(['Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'singer_id'])
[tensor([[0.4855],
        [0.4850],
        [0.4815],
        [0.4826]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5032],
        [0.5026],
        [0.5056],
        [0.5036]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5086],
        [0.5092],
        [0.5077],
        [0.5081]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5052],
        [0.5060],
        [0.5034],
        [0.5047]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5009],
    

In [22]:
optimizer.zero_grad()
y.loss.backward()# retain_graph=True
optimizer.step()

In [23]:
print(model)

HuBERTFeatureFusion(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbeddi

In [24]:
print(model.classifiers['Pitch'].weight.grad)
# print(model.projectors['F4_Hz'].weight)

tensor([[ 4.7313e-03,  2.7105e-03, -2.7535e-05, -4.7179e-05,  8.1642e-04,
         -2.6611e-05,  8.4549e-04,  1.6355e-03,  1.0608e-02,  5.9157e-05,
          9.4235e-03, -3.6725e-05, -4.1172e-05, -1.0391e-04, -1.7174e-05,
         -1.9976e-05,  4.7302e-04,  4.6526e-04,  5.8301e-03,  1.8470e-03,
          8.1306e-06,  2.4033e-04,  2.2165e-03, -4.5151e-05,  8.2584e-05,
          4.9126e-03,  4.6617e-04,  2.4454e-04,  2.6322e-03,  1.9220e-03,
          5.2963e-04,  5.3123e-03,  1.7930e-04,  1.3687e-03, -3.8049e-05,
          8.3580e-03, -2.8608e-05,  2.1641e-04,  9.1061e-03,  5.6752e-03,
          8.6914e-03,  4.2083e-03,  5.0129e-03,  2.0816e-03,  7.0089e-03,
          2.3069e-04, -4.8539e-05,  5.9818e-04, -7.4531e-05,  7.6400e-03,
         -3.8980e-05,  5.9020e-03, -2.3143e-05,  3.6540e-03, -6.0976e-05,
          1.8778e-03,  2.9130e-03, -1.5325e-04, -6.7737e-05,  9.2217e-04,
         -6.2529e-05,  1.5956e-03, -7.2357e-05, -5.4847e-05,  5.3655e-03,
         -9.7664e-05,  4.3854e-03, -4.

In [25]:
print(model.hubert.encoder.layers[11].final_layer_norm.weight.grad)
print(model.hubert.encoder.layers[11].final_layer_norm.weight)

None
Parameter containing:
tensor([0.2295, 0.3408, 0.2241, 0.2180, 0.3718, 0.2739, 0.4177, 0.7271, 0.2072,
        0.2150, 0.2554, 0.2473, 0.2111, 0.7178, 0.1851, 0.2050, 0.4814, 0.2057,
        0.2379, 0.1952, 0.5957, 0.3611, 0.2522, 0.5278, 0.1882, 1.0547, 0.2152,
        0.4490, 0.4299, 0.2314, 0.2019, 0.2072, 0.2798, 0.2083, 0.4707, 0.2754,
        0.2136, 0.6519, 0.2372, 0.2583, 0.2527, 0.2141, 0.2001, 0.3301, 0.2196,
        0.2537, 0.2581, 0.2908, 0.2773, 0.2332, 0.1733, 0.2766, 0.2544, 0.2703,
        0.2329, 0.2493, 0.2891, 0.2959, 0.2852, 0.6724, 0.2040, 0.2484, 0.2087,
        0.3169, 0.2595, 0.1816, 0.2347, 0.7002, 0.2201, 0.2742, 0.2114, 0.2729,
        0.6841, 0.6235, 0.4590, 0.1656, 0.2288, 0.4587, 0.3438, 0.2225, 0.4719,
        0.2688, 0.2349, 0.2260, 0.2186, 0.2642, 0.2301, 0.3638, 0.2355, 0.4534,
        0.1997, 0.1818, 0.1923, 0.2277, 0.7339, 0.2678, 0.2124, 0.2478, 0.2356,
        0.3901, 0.2098, 0.2205, 0.2212, 0.6611, 0.2272, 0.2209, 0.2822, 0.2277,
        0.570

In [26]:
print(len(model.hubert.encoder.layers))

12


In [27]:
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight.grad)
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight)
# print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

None
Parameter containing:
tensor([[-0.0309, -0.0560, -0.0944,  ..., -0.0551,  0.0054, -0.1141],
        [-0.0452,  0.0558, -0.0271,  ..., -0.0301, -0.0216,  0.0928],
        [ 0.0855, -0.0440,  0.1083,  ..., -0.0047,  0.0585, -0.1077],
        ...,
        [-0.0461, -0.0809,  0.0314,  ...,  0.0263,  0.0117,  0.0425],
        [-0.0258, -0.0419, -0.0305,  ..., -0.0015, -0.0509, -0.1317],
        [-0.0070, -0.0095,  0.0326,  ...,  0.0144,  0.0219, -0.0197]],
       device='cuda:0')


In [28]:
for i in range(50):
    y = model(
        audio_normalized=b[0]['input_values'],
        attention_mask=b[0]['attention_mask'],
        labels=b[1],
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True
    )
    print(y.loss)
    # print(model.classifiers['F4_Hz'].weight)
    optimizer.zero_grad()
    y.loss.backward()
    optimizer.step()

tensor(3.4129, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7018, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1417, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1420, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0910, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.9878, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.9223, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8305, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7518, device='cuda:0', grad_fn=<AddBackward0>)


tensor(1.6777, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6005, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.5156, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4450, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3339, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2534, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2174, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.1924, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.1621, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.1320, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.1143, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.1006, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0951, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0835, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0853, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0860, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0806, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0786, device='cuda:0', grad_fn=<AddBack

In [29]:
print(y['logits']['singer_id'][0].argmax(dim=1, keepdim=True).squeeze())

tensor([3, 2, 2, 5], device='cuda:0')
